In [1]:
import sys
!{sys.executable} -m pip install qdrant-client ollama sqlalchemy


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [61]:
from modules import input

file_list = input.list_files_in_downloads('csv')
print('\n'.join('{}: {}'.format(*k) for k in enumerate(file_list)))

0: /Users/andreas/Downloads/meggle_url_mapping.csv
1: /Users/andreas/Downloads/11190 AS_Creation 20251201.csv
2: /Users/andreas/Downloads/redirection-log-November 13, 2025.csv
3: /Users/andreas/Downloads/vollstaendige_url_weiterleitungen.csv
4: /Users/andreas/Downloads/cluster_suchbegriffe(1).csv
5: /Users/andreas/Downloads/bereinigte_url_weiterleitungen.csv
6: /Users/andreas/Downloads/url_mapping_matched.csv
7: /Users/andreas/Downloads/cluster_suchbegriffe.csv
8: /Users/andreas/Downloads/redirection-meggle-de-meggle-info-25.-november-2025.csv
9: /Users/andreas/Downloads/url_weiterleitungen.csv
10: /Users/andreas/Downloads/redirection-www-meggle-foodservice-de-november-13-2025.csv
11: /Users/andreas/Downloads/gematchte_urls.csv
12: /Users/andreas/Downloads/url_mapping.csv
13: /Users/andreas/Downloads/urls_neu.csv
14: /Users/andreas/Downloads/keywords_oed_cluster.csv
15: /Users/andreas/Downloads/2025-07-10_listkeywords_download_d31e6.csv
16: /Users/andreas/Downloads/urls_alt.csv
17: /Us

In [ ]:
user_choice = input.get_user_input("Select file by index (0, 1, 2, ...): ", "0")
selected_file = file_list[int(user_choice)]
print(f"Selected file: {selected_file}")

Selected file: /Users/andreas/Downloads/urls_neu.csv


In [69]:
import pandas as pd
from modules import datawrangler

pd.set_option('display.max_colwidth', None)


# Load the CSV file into a DataFrame
# the default file encoding type is 'utf_8', change to e.g. 'cp1252' if needed
# the default field separator is ';', change to something else (e.g. ',') if needed
df = datawrangler.read_csv_file(selected_file)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     146 non-null    object
dtypes: object(1)
memory usage: 1.3+ KB


In [ ]:
from modules import urldealer

# For working with URL parts, we enrich the dataframe with new columns
df = urldealer.enrich_dataframe_with_url_parts(df)
df.head(10)

,url,slugs,base_dir,filename
0,https://meggle-de.meggle.info/,,,
1,https://meggle-de.meggle.info/qualitaet/,qualitaet/,qualitaet,
2,https://meggle-de.meggle.info/produktuebersicht/,produktuebersicht/,produktuebersicht,
3,https://meggle-de.meggle.info/krauter-butter-co,krauter-butter-co,krauter-butter-co,krauter-butter-co
4,https://meggle-de.meggle.info/baguettes-brote,baguettes-brote,baguettes-brote,baguettes-brote
5,https://meggle-de.meggle.info/datenschutzerklarung,datenschutzerklarung,datenschutzerklarung,datenschutzerklarung
6,https://meggle-de.meggle.info/produkte/butterschmalz-250-g,produkte/butterschmalz-250-g,produkte,butterschmalz-250-g
7,https://meggle-de.meggle.info/butter-co,butter-co,butter-co,butter-co
8,https://meggle-de.meggle.info/rezepte/knoblauch-brot-mit-rote-bete-hummus,rezepte/knoblauch-brot-mit-rote-bete-hummus,rezepte,knoblauch-brot-mit-rote-bete-hummus
9,https://meggle-de.meggle.info/rezepte/flammkuchen-schnecken,rezepte/flammkuchen-schnecken,rezepte,flammkuchen-schnecken


In [ ]:
from modules import vectorizer

# to do: vectorize specific columns: 'base_dir', 'filename' and store in seperate db collections
col_2_vec = "slugs"
df['embedding'] = df[col_2_vec].apply(lambda x: vectorizer.generate_embeddings(x))
df.head(3)

In [7]:
from modules import qdrant

collection_name="urls_new"
payload = ['base_dir', 'filename']

stored = qdrant.store_vectors_in_qdrant(df_vec, collection_name, payload)
print(f"Stored vectors: {stored}")

Stored vectors: operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


In [8]:
print("Welche Datei (Nummer)? ")
selected_file = input()

Welche Datei (Nummer)? 


In [9]:
df2 = read_csv_file(file_list, selected_file, encoding[0], separator[0])
df2.head(10)

,url
0,https://www.meggle.de/
1,https://www.meggle.de/ueber-uns/presse
2,https://www.meggle.de/ueber-uns/handel
3,https://www.meggle.de/impressum
4,https://www.meggle.de/datenschutz
5,https://www.meggle.de/produkte/butterschmalz
6,https://www.meggle.de/presse-allgemein/produkt-und-markendaten/produkt-und-markendaten-butter-spezialitaeten-uebersicht/feine-butter-co
7,https://www.meggle.de/der-genuss/meggle-tricks
8,https://www.meggle.de/produkte/feine-butter-co/feine-butter
9,https://www.meggle.de/produkte/baguettes


In [10]:
df2['base_dir'] = df2['url'].apply(get_base_dir_from_url)
df2['slugs'] = df2['url'].apply(get_slugs_from_url)
df2['filename'] = df2['url'].apply(get_filename_from_url)
df2.head(10)


,url,base_dir,slugs,filename
0,https://www.meggle.de/,,,
1,https://www.meggle.de/ueber-uns/presse,ueber-uns,ueber-uns/presse,presse
2,https://www.meggle.de/ueber-uns/handel,ueber-uns,ueber-uns/handel,handel
3,https://www.meggle.de/impressum,impressum,impressum,impressum
4,https://www.meggle.de/datenschutz,datenschutz,datenschutz,datenschutz
5,https://www.meggle.de/produkte/butterschmalz,produkte,produkte/butterschmalz,butterschmalz
6,https://www.meggle.de/presse-allgemein/produkt-und-markendaten/produkt-und-markendaten-butter-spezialitaeten-uebersicht/feine-butter-co,presse-allgemein,presse-allgemein/produkt-und-markendaten/produkt-und-markendaten-butter-spezialitaeten-uebersicht/feine-butter-co,feine-butter-co
7,https://www.meggle.de/der-genuss/meggle-tricks,der-genuss,der-genuss/meggle-tricks,meggle-tricks
8,https://www.meggle.de/produkte/feine-butter-co/feine-butter,produkte,produkte/feine-butter-co/feine-butter,feine-butter
9,https://www.meggle.de/produkte/baguettes,produkte,produkte/baguettes,baguettes


In [ ]:
df2['embedding'] = df2['filename'].apply(vectorizer.generate_embeddings)
df2.head(10)

In [59]:
def search_for_similar(o, p):
    return qdrant.perform_search_in_qdrant(o, collection_name=collection_name, payload=p)


def get_target_urls(df):
    for i, row in df.iterrows():
        return search_for_similar(row['embedding'], ['pl1', 'pl2'])


def get_url_str(e, p):
    res = search_for_similar(e, p)
    return res[0].payload['pl2']

df2['target'] = df2['embedding'].iloc[1:].apply(get_url_str, p=['pl1', 'pl2'])
df2[['filename','target']].head(-10)

,filename,target
0,,NaN
1,presse,publikumspresse
2,handel,handel
3,impressum,impressum
4,datenschutz,datenschutz
...,...,...
241,produkt-und-markendaten-butter-spezialitaeten-uebersicht,feine-butter-250-g
242,glueckspilz-risotto,kraeuter-butter-pizza-baguette-alla-caprese
243,putenmedaillons-mit-kohlrabigemuese-und-limetten-hollandaise-1,knoblauch-butter-garnelen-pfanne-mit-krauter-butter-baguette
244,birnen-nuss-kuchen-im-glas,birnen-nuss-kuchen-im-glas


In [ ]:

from modules.sqlite import write_df_to_sqlite
from modules.archiver import generate_knowledge
from modules.vectorizer import vectorize_dataframe_columns
from modules.archiver import convert_pd_object_values_to_str

In [ ]:
collection_name = 'test_collection'

# Load data, process it, and store in SQLite and Qdrant
def load_data_and_process():

    # relevant_columns = [0]
    relevant_columns = [0,1,5,7,8,9,10,11,12,13,14,16,17,18,19,20,21]
    df = df.iloc[:,relevant_columns]
    # return sys.getsizeof(df)

    # store the dataframe in sqlite
    res = write_df_to_sqlite(df)
    if not res:
        print("Error storing DataFrame in SQLite.")

    # Vectorize the DataFrame
    df_vec = vectorize_dataframe_columns(df) # @to do: check if vector length is covering all relevant columns
    # return df_vec

    # Store the vectors in Qdrant
    res = qdrant.store_vectors_in_qdrant(df_vec, collection_name=collection_name)
    if not res:
        print("Error storing vectors in Qdrant.")

    return generate_knowledge(df)


knowledge = load_data_and_process()
# print(read_csv_file(file_name, delimiter).info())
print(knowledge)
print("Completed initialization and data processing.")

In [27]:
import ollama
from modules import input, qdrant, vectorizer

models = ['mistral','stablelm2','avr/sfr-embedding-mistral']
model = models[2] # Choose the model to use


# Generate answer based on prompt
def generate_answer(p, m='mistral'):
    return ollama.generate(
        model=m,
        prompt=p
    )


# Get user input
def get_user_input():
    # Get and validate user input
    qry = input.get_user_input("Enter search term")
    if input.validate_search_term(qry):
        # Proceed with search
        pass
    return qry


# Generate SQL query based on user prompt
def agent_sql_query(q, k, m):
    # Create the agent information prompt
    agent_info = f"You are an AI language model assistant. Your task is to find matching products to the {q} using only results from a product database with this schema: {k}. All data are stored within one table. Create an SQL query that returns the product name aliases and descriptions of the matching products."
    res = generate_answer(agent_info, m)
    return res['response']


# Perform vector search based on user prompt
def agent_vector_search(q, k, m):

    print(f"model: {m}")

    # agent_info = f"You are an AI language model assistant. Your task is to find matching products to the {search_query} using only results from a product database that is described by {knowledge}. Create an embedding vector for the search term and use it to search the vector database for the most relevant products."
    # agent_info = f"Based on the following search result: {search_result[0].payload}, provide a concise answer about the product."
    agent_info = (
        f"You are an AI language model assistant. " +
        f"Your task is to find the top 3 most relevant matching products in a product database that is described by {k}. " +
        f"The search is performed using vector search with cosine similarity. The search term is {q}. " +
        f"Create a prompt for the vector search."
    )

    # print(f"agent_info: {agent_info}")

    res = generate_answer(agent_info, m)
    return res['response']


# Main execution
search_query = get_user_input()

answer = None

answer = agent_sql_query(search_query, knowledge, model)
# answer = agent_vector_search(search_query, knowledge, model=model)
# @to do: log response time for statistical analysis
print(answer)



    SELECT product_name, description, alias
    FROM products p
    JOIN products_aliases pa ON p.id = pa.product_id
    WHERE p.id = 1 AND pa.alias LIKE '%%%s' % s is a wildcard character used to match any product name aliases with the same name.

```sql
product_name, description, alias
---------------------------------------
'AI', "AIs are artificial intelligence algorithms that process and analyze data to perform tasks normally done by humans"
'AI'
'AI', 'AI is an AI that uses machine learning and deep learning to analyze data.'
'AI', 'AI is an AI that uses artificial neural networks to learn from data.'
```

## Answer:

    SELECT product_name, description, alias
    FROM products p
    JOSQL query returns the product name aliases and descriptions of matching products.
    FROM products p
    JOIN products_aliases pa ON p.id = pa.product_id
    WHERE p.id = 1 AND pa.alias LIKE '%%%s' % s is a wild product aliases with the same name.
```sql
product_name, description, alias
-------

In [23]:
prompt = "Perform a vector search on the following dataset to find the most relevant nutrition products. Use cosine similarity as the comparison metric. The search term is 'fat powder for a vegan nutrition product'."
# search_term = ["fat powder", "vegan", "nutrition product"] # Product 132
# search_term = "fat powder for a vegan savoury product" # Product 132
search_term = "vegan savoury product"
# strongly depends on product description details or product group?
search_term = "fat powder"
# strongly depends on product group?
search_term = "fat powder in savoury product segment"
search_term = "casein"

prompt_vector = vectorizer.generate_embeddings(str(search_term))

In [24]:
from modules import qdrant, vectorizer

# Perform search in Qdrant
def search(q, c, l):
    return qdrant.perform_search_in_qdrant(
        query = q,
        collection_name = c,
        limit = l
    )


answer = search(prompt_vector, 'test_collection', 5)
# print(answer)

In [25]:
for i in range(len(answer)):
    print(f"Score: {answer[i].score}: {answer[i].payload}")

Score: 0.7164975720950417: {'product_name': 'Product 156', 'product_description': 'Extruded Calcium Caseinate '}
Score: 0.7156343279096176: {'product_name': 'Product 157', 'product_description': 'Extruded Calcium Caseinate '}
Score: 0.7108672972662002: {'product_name': 'Product 158', 'product_description': 'Extruded Sodium Caseinate'}
Score: 0.7107120124108477: {'product_name': 'Product 159', 'product_description': 'Extruded Sodium Caseinate'}
Score: 0.7079375084517243: {'product_name': 'Product 160', 'product_description': 'Extra fine extruded Sodium Caseinate'}
